In [1]:
import numpy as np
from scipy import stats
import math
import pandas as pd

In [2]:
pathToNoDelimiter = '|'

In [3]:
# Example PN-Tree
PN_Types_Net = []
for eachPN in open('pn_types_net_test.txt').readlines():
    if len(eachPN) > 0 and pathToNoDelimiter in eachPN:
        PN_Types_Net.append(eachPN.strip())

PN_Types_Net    

['YouLike1 | Exploit | Personalized | RootNode',
 'YouLike2 | Exploit | Personalized | RootNode',
 'BecauseYouLike1 | Explore | Personalized | RootNode',
 'BecauseYouLike2 | Explore | Personalized | RootNode',
 'Supply1 | Non-Personalized | RootNode',
 'Supply2 | Non-Personalized | RootNode']

In [4]:
# Mapping child node to parent node & parent node to child node
ChildParentDict = {}
ParentChildDict = {}
PNs = []

for eachPN in PN_Types_Net:
    path = [eNode.strip() for eNode in eachPN.split(pathToNoDelimiter) if len(eNode) > 1]
    PNs.append(path[0])
    for child, parent in zip(path, path[1:]):
        ChildParentDict[child]  = parent
        if parent in ParentChildDict:
            ParentChildDict[parent].add(child)
        else:
            ParentChildDict[parent] = {child}        

In [5]:
# PN's performance till date
# sub population under consider is PopType1(context), for every context we are going to maintain this dictionary 

PNsAlphaBeta = {}

SubPopulationType = 'Bangalore@City|P1@P*|Mon-Fri@DayCat|Lunch@TimeSlot'
PNsAlphaBeta[SubPopulationType] = {}
PNsAlphaBeta[SubPopulationType]['YouLike1']        = {'clicks': 0,  'impressions': 0}
PNsAlphaBeta[SubPopulationType]['YouLike2']        = {'clicks': 0,    'impressions': 0}
PNsAlphaBeta[SubPopulationType]['BecauseYouLike1'] = {'clicks': 0,  'impressions': 0}
PNsAlphaBeta[SubPopulationType]['BecauseYouLike2'] = {'clicks': 0,   'impressions': 0}
PNsAlphaBeta[SubPopulationType]['Supply1']         = {'clicks': 0,   'impressions': 0}
PNsAlphaBeta[SubPopulationType]['Supply2']         = {'clicks': 0,  'impressions': 0}

PNsAlphaBeta


{'Bangalore@City|P1@P*|Mon-Fri@DayCat|Lunch@TimeSlot': {'YouLike1': {'clicks': 0,
   'impressions': 0},
  'YouLike2': {'clicks': 0, 'impressions': 0},
  'BecauseYouLike1': {'clicks': 0, 'impressions': 0},
  'BecauseYouLike2': {'clicks': 0, 'impressions': 0},
  'Supply1': {'clicks': 0, 'impressions': 0},
  'Supply2': {'clicks': 0, 'impressions': 0}}}

In [6]:
# computing some global variables, which will be used in downstream logic
nodeSet = set()
heightOfTheTree = 1
allPaths = []
for eachPN in PN_Types_Net:
    path = [eNode.strip() for eNode in eachPN.split('|') if len(eNode) > 1]
    for node in path:
        nodeSet.add(node)
    allPaths.append(path) 
    if len(path) > heightOfTheTree:
        heightOfTheTree = len(path)       

In [7]:
# mapping nodeid to node index to index nodeid in matrix
noOfNodes = len(nodeSet)
nodeIdx = noOfNodes
nodeIdToIndex = {}
for idx in range(heightOfTheTree): # At every level PN-Types-Hierachy
    for path in allPaths:          # for every PN
        if len( path[idx:idx+1] ) > 0: # if there is a node in this path @ this hieaarchy level
            if path[idx:idx+1][0] not in nodeIdToIndex:
                nodeIdToIndex[path[idx:idx+1][0]] = nodeIdx
                nodeIdx = nodeIdx - 1
print('No Of Nodes in our reference tree:\t', noOfNodes)
print('nodeIdToIndex mapping:\t', nodeIdToIndex)             

No Of Nodes in our reference tree:	 11
nodeIdToIndex mapping:	 {'YouLike1': 11, 'YouLike2': 10, 'BecauseYouLike1': 9, 'BecauseYouLike2': 8, 'Supply1': 7, 'Supply2': 6, 'Exploit': 5, 'Explore': 4, 'Non-Personalized': 3, 'Personalized': 2, 'RootNode': 1}


In [8]:
# clicks ( sum of every column in resultant's matrix corresponds to total clicks in that branch )
clicks = np.zeros ( (len(nodeIdToIndex), len(nodeIdToIndex) ), dtype=int)
nodeIdToIndex_items = list(nodeIdToIndex.items())
nodeIdToIndex_items.sort(key=lambda x:x[1], reverse=True)

# for all terminal nodes
for path in allPaths:
    clicks[nodeIdToIndex[path[0]]-1][nodeIdToIndex[path[0]]-1] = PNsAlphaBeta[SubPopulationType][ path[0] ]['clicks']    

# total clicks @ every non-terminal nodes
for path in allPaths:
    for child, parent in zip(path, path[1:]):        
        childIdx  = nodeIdToIndex[child]-1
        parentIdx = nodeIdToIndex[ChildParentDict[child]]-1
        clicks[ childIdx ][ parentIdx ] = sum(clicks[:, childIdx ])

In [9]:
cols = [item[0] for item in nodeIdToIndex_items][::-1]
pd.DataFrame(clicks, columns=cols, index=cols).to_csv('/Users/viswanath.g/Desktop/clicks.csv')

In [10]:
pd.read_csv('/Users/viswanath.g/Desktop/clicks.csv')

,Unnamed: 0,RootNode,Personalized,Non-Personalized,Explore,Exploit,Supply2,Supply1,BecauseYouLike2,BecauseYouLike1,YouLike2,YouLike1
0,RootNode,0,0,0,0,0,0,0,0,0,0,0
1,Personalized,0,0,0,0,0,0,0,0,0,0,0
2,Non-Personalized,0,0,0,0,0,0,0,0,0,0,0
3,Explore,0,0,0,0,0,0,0,0,0,0,0
4,Exploit,0,0,0,0,0,0,0,0,0,0,0
5,Supply2,0,0,0,0,0,0,0,0,0,0,0
6,Supply1,0,0,0,0,0,0,0,0,0,0,0
7,BecauseYouLike2,0,0,0,0,0,0,0,0,0,0,0
8,BecauseYouLike1,0,0,0,0,0,0,0,0,0,0,0
9,YouLike2,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# impressions ( sum of every column in resultant's matrix corresponds to total impressions in that branch )
impressions = np.zeros ( (len(nodeIdToIndex), len(nodeIdToIndex) ), dtype=int)
nodeIdToIndex_items = list(nodeIdToIndex.items())
nodeIdToIndex_items.sort(key=lambda x:x[1], reverse=True)

# all terminal nodes
for path in allPaths:
    if len(path) > 0:
        impressions[nodeIdToIndex[path[0]]-1][nodeIdToIndex[path[0]]-1] = PNsAlphaBeta[SubPopulationType][ path[0] ]['impressions']    

# total impressions @ every non-terminal nodes
for path in allPaths:
    if len(path) > 0:
        for child, parent in zip(path, path[1:]):        
            childIdx  = nodeIdToIndex[child]-1
            parentIdx = nodeIdToIndex[ChildParentDict[child]]-1
            impressions[ childIdx ][ parentIdx ] = sum(impressions[:, childIdx ])

In [12]:
cols = [item[0] for item in nodeIdToIndex_items][::-1]
pd.DataFrame(impressions, columns=cols, index=cols).to_csv('/Users/viswanath.g/Desktop/impressions.csv')
pd.read_csv('/Users/viswanath.g/Desktop/impressions.csv')


,Unnamed: 0,RootNode,Personalized,Non-Personalized,Explore,Exploit,Supply2,Supply1,BecauseYouLike2,BecauseYouLike1,YouLike2,YouLike1
0,RootNode,0,0,0,0,0,0,0,0,0,0,0
1,Personalized,0,0,0,0,0,0,0,0,0,0,0
2,Non-Personalized,0,0,0,0,0,0,0,0,0,0,0
3,Explore,0,0,0,0,0,0,0,0,0,0,0
4,Exploit,0,0,0,0,0,0,0,0,0,0,0
5,Supply2,0,0,0,0,0,0,0,0,0,0,0
6,Supply1,0,0,0,0,0,0,0,0,0,0,0
7,BecauseYouLike2,0,0,0,0,0,0,0,0,0,0,0
8,BecauseYouLike1,0,0,0,0,0,0,0,0,0,0,0
9,YouLike2,0,0,0,0,0,0,0,0,0,0,0


In [13]:
# # sampling ctrs for every terminal of the node ( which is nothing but every PN)
# # simple average 

# sampled_cts = {}
# for path in allPaths:
#     sampled_ctrs_per_path = []
#     child = path[0]
    
#     childIdx  = nodeIdToIndex[child]-1 
#     alpha_    = sum(clicks[:,childIdx])
#     beta_     = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])
    
#     sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0] )
    
#     while child in ChildParentDict:
#         child  = ChildParentDict[ child ]
#         childIdx  = nodeIdToIndex[child]-1 
#         alpha_ = sum(clicks[:,childIdx])
#         beta_  = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])        
#         sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0] )
#     sampled_cts[path[0]] = sum(sampled_ctrs_per_path) / len(sampled_ctrs_per_path)
    
# sampled_cts    

In [14]:
# # sampling ctrs for every terminal of the node ( which is nothing but every PN)
# # with geometric weight decays
# lambda_ = 0.75
# sampled_cts = {}

# terminalToRefParntNodeDist = 0

# for path in allPaths[4:5]:
#     weights = []
#     sampled_ctrs_per_path = []
    
#     child = path[0]    
#     childIdx  = nodeIdToIndex[child]-1 
#     alpha_ = sum(clicks[:,childIdx])
#     beta_  = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])
#     sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0]   )
#     weights.append( math.pow(lambda_, terminalToRefParntNodeDist) )
    
#     while child in ChildParentDict:
#         child  = ChildParentDict[ child ]
#         childIdx  = nodeIdToIndex[child]-1 
#         alpha_ = sum(clicks[:,childIdx])
#         beta_  = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])        
                
#         sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0]   )
#         terminalToRefParntNodeDist += 1
#         weights.append( math.pow(lambda_, terminalToRefParntNodeDist)  )
#     print(weights)

#     normalized_weights = [ weight_ / sum(weights) for weight_ in weights]
#     sampled_ctrs_per_path_nw = [ weight_ * sample_ctr for weight_, sample_ctr in zip(normalized_weights, sampled_ctrs_per_path)]
    
#     sampled_cts[path[0]] = sum( sampled_ctrs_per_path_nw ) 

# print(sampled_cts)    
# sampled_ctrs_per_path, normalized_weights

In [15]:
# # sampling ctrs for every terminal of the node ( which is nothing but every PN)
# # with exponential weight decays

# expDecayFact = 1
# sampled_cts = {}


# for path in allPaths[0:2]:

#     weights = []
#     posAdj = []
#     sampled_ctrs_per_path = []
#     terminalToRefParntNodeDist = 0
    
#     child = path[0]    
#     childIdx  = nodeIdToIndex[child]-1 
#     alpha_ = sum(clicks[:,childIdx])
#     beta_  = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])
#     sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0]   )
#     posAdj.append(terminalToRefParntNodeDist)
    
#     while child in ChildParentDict:
#         child  = ChildParentDict[ child ]
#         childIdx  = nodeIdToIndex[child]-1 
#         alpha_ = sum(clicks[:,childIdx])
#         beta_  = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])                        
#         sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0]   )
#         terminalToRefParntNodeDist += 1
#         posAdj.append(terminalToRefParntNodeDist)
    
#     weights = [math.exp(-1 * expDecayFact * posImp) for posImp in posAdj]
#     normalized_weights = [ weight_ / sum(weights) for weight_ in weights]
#     sampled_ctrs_per_path_nw = [ weight_ * sample_ctr for weight_, sample_ctr in zip(normalized_weights, sampled_ctrs_per_path)]    
#     sampled_cts[path[0]] = sum( sampled_ctrs_per_path_nw ) 
# print(sampled_cts)    

In [16]:
# # sampling ctrs for every terminal of the node ( which is nothing but every PN)
# # with exponential weight decays

# expDecayFact = 2
# sampled_cts = {}


# for path in allPaths[0:2]:

#     terminalToRefParntNodeDist = 0
    
#     sampled_ctrs_per_path = []
#     posAdj = []    
#     howMuchAParentKnowAboutAChild=[]
    
#     child = path[0]    
#     childIdx  = nodeIdToIndex[child]-1 
#     alpha_ = sum(clicks[:,childIdx])
#     beta_  = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])
#     sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0]   )
#     howMuchAParentKnowAboutAChild.append(alpha_+beta_)    
#     posAdj.append(terminalToRefParntNodeDist)

#     while child in ChildParentDict:        
#         child  = ChildParentDict[ child ]
#         childIdx  = nodeIdToIndex[child]-1 
#         alpha_ = sum(clicks[:,childIdx])
#         beta_  = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])                        
#         sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0]   )
#         howMuchAParentKnowAboutAChild.append(alpha_+beta_)
#         terminalToRefParntNodeDist += 1
#         posAdj.append(terminalToRefParntNodeDist)
    
#     weightImpFromParentToChild = [1.0] + [ 1-(chiImprs/parImprs) for chiImprs, parImprs in zip(howMuchAParentKnowAboutAChild, howMuchAParentKnowAboutAChild[1:])]
#     weights = [ math.exp(-1*expDecayFact * parentToChildImp * posImp) for (posImp, parentToChildImp) in zip(posAdj, weightImpFromParentToChild) ]
#     normalized_weights = [ weight_ / sum(weights) for weight_ in weights]    
#     sampled_ctrs_per_path_nw = [ weight_ * sample_ctr for weight_, sample_ctr in zip(normalized_weights, sampled_ctrs_per_path)]
#     sampled_cts[path[0]] = sum( sampled_ctrs_per_path_nw ) 
    
#     print(weightImpFromParentToChild)
#     print(weights)
#     print(normalized_weights)
#     print(sampled_ctrs_per_path)
    

# print(sampled_cts)    

In [27]:
# sampling ctrs for every terminal of the node ( which is nothing but every PN)
# with exponential weight decays

minSamples = 10
expDecayFact = 2
sampled_cts = {}


for path in allPaths:

    terminalToRefParntNodeDist = 0
    
    sampled_ctrs_per_path = []
    posAdj = []    
    howMuchAParentKnowAboutAChild=[]
    
    child = path[0]    
    childIdx  = nodeIdToIndex[child]-1 
    alpha_ = sum(clicks[:,childIdx])
    beta_  = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])
    if alpha_ + beta_ > minSamples:
        sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0]   )
        howMuchAParentKnowAboutAChild.append(alpha_+beta_)    
        posAdj.append(terminalToRefParntNodeDist)
        terminalToRefParntNodeDist += 1
        
    while child in ChildParentDict:        
        child  = ChildParentDict[ child ]
        childIdx  = nodeIdToIndex[child]-1 
        alpha_ = sum(clicks[:,childIdx])
        beta_  = sum(impressions[:,childIdx]) - sum(clicks[:,childIdx])            
        
        if alpha_ + beta_ > minSamples:
            sampled_ctrs_per_path.append( stats.beta.rvs(alpha_, beta_, size=1)[0]   )
            howMuchAParentKnowAboutAChild.append(alpha_+beta_)            
            posAdj.append(terminalToRefParntNodeDist)
            terminalToRefParntNodeDist += 1
    
    weightImpFromParentToChild = [1.0] + [ chiImprs/parImprs for chiImprs, parImprs in zip(howMuchAParentKnowAboutAChild, howMuchAParentKnowAboutAChild[1:])]
    weights = [ math.exp(-1*expDecayFact * (1-parentToChildImp) * posImp) for (posImp, parentToChildImp) in zip(posAdj, weightImpFromParentToChild) ]
    normalized_weights = [ weight_ / sum(weights) for weight_ in weights]    
    sampled_ctrs_per_path_nw = [ weight_ * sample_ctr for weight_, sample_ctr in zip(normalized_weights, sampled_ctrs_per_path)]
    

    if len(sampled_ctrs_per_path_nw) == 0:
        sampled_ctrs_per_path_nw.append( stats.beta.rvs(10, 990, size=1)[0] ) # 1% ctr
    
    sampled_cts[path[0]] = sum( sampled_ctrs_per_path_nw ) 
#     print(weightImpFromParentToChild)
#     print(weights)
#     print(normalized_weights)
    print(sampled_ctrs_per_path_nw)
    

print(sampled_cts)    

[0.015765388950916212]
[0.018691603938870895]
[0.007697356627616843]
[0.012962491476240678]
[0.008900481313349836]
[0.005588499923225301]
{'YouLike1': 0.015765388950916212, 'YouLike2': 0.018691603938870895, 'BecauseYouLike1': 0.007697356627616843, 'BecauseYouLike2': 0.012962491476240678, 'Supply1': 0.008900481313349836, 'Supply2': 0.005588499923225301}


In [18]:
sum([])

0